In [1]:
import os
import pandas as pd
import numpy as np

os.listdir()

['.ipynb_checkpoints',
 'BTC_USDT.csv',
 'Data pre-processing.ipynb',
 'DATA_Intergration.ipynb',
 'data_out.ipynb',
 'ETH_USDT.csv',
 'LTC_USDT.csv',
 'test2.ipynb',
 'test3_0630_activation.ipynb',
 'test3_0630_latent_dim.ipynb',
 'test6_0712_best.ipynb',
 'vae practice-Copy2.ipynb',
 'XLM_USDT.csv',
 'XRP_USDT.csv',
 '데이터 재구조화.ipynb']

In [43]:
FILENAME='XRP_USDT.csv'

In [44]:
df = pd.read_csv(FILENAME)

In [45]:
df=df.iloc[:,2:]

In [46]:
df

,date_time,open,high,low,close,volume
0,2018-05-04 08:11:00,0.50000,0.50000,0.50000,0.50000,109.51
1,2018-05-04 08:12:00,0.92999,0.92999,0.85000,0.90000,167543.47
2,2018-05-04 08:13:00,0.90000,0.93000,0.89001,0.91000,179750.59
3,2018-05-04 08:14:00,0.91000,1.50000,0.90009,0.92999,107222.48
4,2018-05-04 08:15:00,0.92999,0.95001,0.91020,0.91020,171304.56
...,...,...,...,...,...,...
1530222,2021-03-31 23:55:00,0.57028,0.57029,0.56915,0.56960,218441.60
1530223,2021-03-31 23:56:00,0.56960,0.56987,0.56853,0.56874,384596.60
1530224,2021-03-31 23:57:00,0.56875,0.57048,0.56852,0.56982,206806.60
1530225,2021-03-31 23:58:00,0.56982,0.56999,0.56900,0.56958,234160.50


In [47]:
df.columns = ["Time","Open","High","Low","Close","Volume"]

In [48]:
df

,Time,Open,High,Low,Close,Volume
0,2018-05-04 08:11:00,0.50000,0.50000,0.50000,0.50000,109.51
1,2018-05-04 08:12:00,0.92999,0.92999,0.85000,0.90000,167543.47
2,2018-05-04 08:13:00,0.90000,0.93000,0.89001,0.91000,179750.59
3,2018-05-04 08:14:00,0.91000,1.50000,0.90009,0.92999,107222.48
4,2018-05-04 08:15:00,0.92999,0.95001,0.91020,0.91020,171304.56
...,...,...,...,...,...,...
1530222,2021-03-31 23:55:00,0.57028,0.57029,0.56915,0.56960,218441.60
1530223,2021-03-31 23:56:00,0.56960,0.56987,0.56853,0.56874,384596.60
1530224,2021-03-31 23:57:00,0.56875,0.57048,0.56852,0.56982,206806.60
1530225,2021-03-31 23:58:00,0.56982,0.56999,0.56900,0.56958,234160.50


In [49]:
def preprocess_ratio(data):
    windows = [5,10,20,60,120]
    for window in windows:
        data['close_ma{}'.format(window)] =  data['Close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = data['Volume'].rolling(window).mean()
        data['close_ema{}'.format(window)] =  data['Close'].ewm(span=window).mean()
        data['volume_ema{}'.format(window)] = data['Volume'].ewm(span=window).mean()
        data['close_ma%d_ratio' % window] = (data['Close'] - data['close_ma%d' % window]) / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = (data['Volume'] - data['volume_ma%d' % window]) / data['volume_ma%d' % window]
        data['close_ema%d_ratio' % window] = (data['Close'] - data['close_ema%d' % window]) / data['close_ema%d' % window]
        data['volume_ema%d_ratio' % window] = (data['Volume'] - data['volume_ema%d' % window]) / data['volume_ema%d' % window]
        
    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = (data['Open'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['high_close_ratio'] = (data['High'].values - data['Close'].values) / data['Close'].values
    
    data['low_close_ratio'] = (data['Low'].values - data['Close'].values) / data['Close'].values
    
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = (data['Close'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = (data['Volume'][1:].values - data['Volume'][:-1].values) / data['Volume'][:-1].replace(to_replace=0, method='ffill').replace(to_replace=0, method='bfill').values
    
    shortema = data['Close'].ewm(span=12).mean()
    longema = data['Close'].ewm(span=26).mean()
    DIF = shortema - longema
    DEA = DIF.ewm(span=9).mean()
    data['MACD'] = 2*(DIF-DEA)
    
    delta = data['Close'].diff(1)
    delta = delta[1:]
    
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    
    avg_gain = data['up'].rolling(window=14).mean()
    avg_loss = abs(data['down'].rolling(window=14).mean())
    RS = avg_gain / avg_loss
    
    RSI = 100.0-(100.0/(1.0+RS))
    data['RSI'] = RSI
    
    ndays_high=data["High"].rolling(window=14, min_periods=1).max()
    ndays_low=data['Low'].rolling(window=14, min_periods=1).min()
    fast_k = ((data["Close"]-ndays_low)/(ndays_high-ndays_low))*100
    data["Fast_k"]=fast_k
    
    typical_price = ((df['Close'] + df['High'] + df['Low']) / 3)
    money_flow = typical_price * df['Volume']
    positive_flow = []
    negative_flow = []
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1] :
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    MF_period = 14
    positive_mf = [np.nan]*14
    negative_mf = [np.nan]*14
    for i in range(MF_period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1-MF_period : i+1]))
        
    for i in range(MF_period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1-MF_period : i+1]))
        
    mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))
    df['MFI'] = mfi
    
    data=data.drop(columns=['up','down'])
    data=data.dropna(axis=0)
    
    return data

## 이전버전

In [50]:
def preprocess(data):
    windows = [5,10,20,60,120]
    for window in windows:
        data['close_ma{}'.format(window)] =  data['Close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = data['Volume'].rolling(window).mean()
        data['close_ema{}'.format(window)] =  data['Close'].ewm(span=window).mean()
        data['volume_ema{}'.format(window)] = data['Volume'].ewm(span=window).mean()

        
    shortema = data['Close'].ewm(span=12).mean()
    longema = data['Close'].ewm(span=26).mean()
    DIF = shortema - longema
    DEA = DIF.ewm(span=9).mean()
    data['MACD'] = 2*(DIF-DEA)
    
    delta = data['Close'].diff(1)
    delta = delta[1:]
    
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    
    avg_gain = data['up'].rolling(window=14).mean()
    avg_loss = abs(data['down'].rolling(window=14).mean())
    RS = avg_gain / avg_loss
    
    RSI = 100.0-(100.0/(1.0+RS))
    data['RSI'] = RSI
    

    ndays_high=data["High"].rolling(window=14, min_periods=1).max()
    ndays_low=data['Low'].rolling(window=14, min_periods=1).min()
    fast_k = ((data["Close"]-ndays_low)/(ndays_high-ndays_low))*100
    data["Fast_k"]=fast_k

    typical_price = ((data['Close'] + data['High'] + data['Low']) / 3)
    money_flow = typical_price * data['Volume']
    positive_flow = []
    negative_flow = []
    
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1] :
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            
            positive_flow.append(0)
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    MF_period = 14
    positive_mf = [np.nan]*14
    negative_mf = [np.nan]*14
    for i in range(MF_period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1-MF_period : i+1]))
        
    for i in range(MF_period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1-MF_period : i+1]))
        
    mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))
    data['MFI'] = mfi

    data=data.drop(columns=['up','down'])
    data=data.dropna(axis=0)
    
    return data

In [51]:
temp=preprocess_ratio(df)

In [52]:
temp.to_csv(FILENAME, mode='w',index=False)

In [42]:
temp.head(2)

,Time,Open,High,Low,Close,Volume,close_ma5,volume_ma5,close_ema5,volume_ema5,...,volume_ema120_ratio,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,MACD,RSI,Fast_k,MFI
119,2018-05-31 11:36:00,0.29039,0.29039,0.29039,0.29039,8844.00,0.290468,3113.762,0.290445,4631.48887,...,0.342261,0.000000,0.000000,0.000000,0.000000,1.445120,-0.000136,51.449275,9.482759,50.693372
120,2018-05-31 11:37:00,0.29029,0.29132,0.29029,0.29131,7937.44,0.290574,4301.088,0.290734,5733.47258,...,0.199989,-0.000344,0.000034,-0.003501,0.003168,-0.102506,-0.000017,63.586957,88.793103,61.592458
